In [ ]:
import analysis.aggregator as ag
import data.file_handler as fh
from analysis.pipeline_blocks import evaluate_clustering
from data.corpus_handler import CorpusName, CorpusHandler

In [ ]:
ABS_PATH = fh.add_and_get_abs_path('./data/results')
dictionary = fh.load_df(ABS_PATH, 'Toy-linkage_complete-dist_0.4-dictionary.pkl')
#dictionary = fh.load_df(ABS_PATH, 'SemCor-dist_0.4-dictionary.pkl')

In [ ]:
ag.collect_references_and_word_vectors(
    ag.unpack_per_word_vector(
        dictionary,
        ['reference_id', 'word_vector_id', 'sense']),
    ['token', 'sense'])

## Corpus Baseline

In [ ]:
corpus = CorpusHandler(CorpusName.SEMCOR, './data/corpus_cache')
tagged_tokens = corpus.get_tagged_tokens()

In [ ]:
ag.calc_corpus_statistics(corpus)

One cluster/sense per token

In [ ]:
tagged_tokens['sense'] = range(tagged_tokens.token.count())
tagged_tokens.sense.nunique(), evaluate_clustering(corpus, tagged_tokens)

One cluster/sense per unique token

In [ ]:
tagged_tokens['sense'] = tagged_tokens.token.factorize()[0]
tagged_tokens.sense.nunique(), evaluate_clustering(corpus, tagged_tokens)